In [1]:
from SRCNN_different_specs import RunSRCNN, SRCNN
from custom_image_dataset import CustomImageDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch
from torch import nn

In [2]:
DATASET_ROOT="./datasets/"
DATASET_NAME = "Flickr2K"

In [3]:
%%time
transform_x = transforms.Compose([
    transforms.Resize((11, 11), interpolation=Image.BICUBIC),
    transforms.Resize((33, 33), interpolation=Image.BICUBIC)
])

transform_y = transforms.Compose([
    transforms.CenterCrop((33, 33))
])

# Set up the data loaders
train_data_set = CustomImageDataset(
    img_dir=f"{DATASET_ROOT}/{DATASET_NAME}_train_unfiltered", transform=transform_x, target_transform=transform_y)
validation_data_set = CustomImageDataset(
    img_dir=f"{DATASET_ROOT}/{DATASET_NAME}_validation_unfiltered", transform=transform_x, target_transform=transform_y)

# Report split sizes
print('Training set has {} instances'.format(len(train_data_set)))
print('Validation set has {} instances'.format(len(validation_data_set)))

100%|██████████| 2650/2650 [00:39<00:00, 67.32it/s]


Training set has 159000 instances
Validation set has 53000 instances
CPU times: total: 14.3 s
Wall time: 3min 42s


In [4]:
%%time

# https://towardsdatascience.com/7-tips-for-squeezing-maximum-performance-from-pytorch-ca4a40951259
pin_memory = False
batch_size = round(0.35*(2**13))

train_dataloader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True, pin_memory=pin_memory) # TODO: aumentar batch_size p/ 2^12
validation_dataloader = DataLoader(validation_data_set, batch_size=batch_size, pin_memory=pin_memory)

CPU times: total: 14.3 s
Wall time: 36.5 s


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-3

model_f2_5 = SRCNN(f2=5).to(device)
optimizer_f2_5 = torch.optim.Adam(model_f2_5.parameters(), lr=lr)
scheduler_f2_5 = torch.optim.lr_scheduler.LinearLR(
    optimizer_f2_5,
    start_factor=1.0,
    end_factor=0.01,
    total_iters=60)

model_f2_1 = SRCNN(f2=1).to(device)
optimizer_f2_1 = torch.optim.Adam(model_f2_1.parameters(), lr=lr)
scheduler_f2_1 = torch.optim.lr_scheduler.LinearLR(
    optimizer_f2_1,
    start_factor=1.0,
    end_factor=0.01,
    total_iters=60)


In [6]:
run_srcnn = RunSRCNN()

In [7]:
%%time
run_srcnn.train(
    model=model_f2_1,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    optimizer=optimizer_f2_1,
    epochs=100,
    scheduler=scheduler_f2_1,
    loss_fn=nn.MSELoss(),
)


epoch 0
-------------------------------
Training on 159000 samples...
Batch size: 2867
Number of batches: 56


  0%|          | 0/56 [00:00<?, ?it/s]C:\Users\Pedro\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 56/56 [04:13<00:00,  4.52s/it]


Training Error: 
 Avg loss: 0.043581 



100%|██████████| 19/19 [00:45<00:00,  2.37s/it]


Validation Error: 
 Avg loss: 0.015762 

Learning rate (antes): 0.001
Learning rate (depois): 0.0009835

epoch 1
-------------------------------
Training on 159000 samples...
Batch size: 2867
Number of batches: 56


...

Validation Error: 
 Avg loss: 0.003471 

Learning rate (antes): 1.0000000000000028e-05
Learning rate (depois): 1.0000000000000028e-05

epoch 99
-------------------------------
Training on 159000 samples...
Batch size: 2867
Number of batches: 56


100%|██████████| 56/56 [02:48<00:00,  3.01s/it]


Training Error: 
 Avg loss: 0.003474 



100%|██████████| 19/19 [00:40<00:00,  2.13s/it]

Validation Error: 
 Avg loss: 0.003471 

Learning rate (antes): 1.0000000000000028e-05
Learning rate (depois): 1.0000000000000028e-05
CPU times: total: 5h 40min 4s
Wall time: 7h 39min 11s


In [8]:
df_f2_1 = run_srcnn.get_model_df()
df_f2_1.tail()

,epoch,time,lr,train_loss,validation_loss
95,95,273.364173,0.00001,0.003477,0.003472
96,96,291.091623,0.00001,0.003477,0.003472
97,97,270.311664,0.00001,0.003475,0.003472
98,98,267.328737,0.00001,0.003473,0.003471
99,99,270.157735,0.00001,0.003474,0.003471


In [9]:
run_srcnn.save_model_df('./results/srcnn/dataframes/model_f2_1_unfiltered.csv')
run_srcnn.save_model_weights(model_f2_1, './results/srcnn/dataframes/model_f2_1_unfiltered.pth')

In [10]:
%%time

run_srcnn.clear_all()

run_srcnn.train(
    model=model_f2_5,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    optimizer=optimizer_f2_5,
    epochs=100,
    scheduler=scheduler_f2_5,
    loss_fn=nn.MSELoss(),
)


epoch 0
-------------------------------
Training on 159000 samples...
Batch size: 2867
Number of batches: 56


  0%|          | 0/56 [00:00<?, ?it/s]C:\Users\Pedro\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 56/56 [03:55<00:00,  4.20s/it]


Training Error: 
 Avg loss: 0.025352 



100%|██████████| 19/19 [00:45<00:00,  2.41s/it]


Validation Error: 
 Avg loss: 0.006658 

Learning rate (antes): 0.001
Learning rate (depois): 0.0009835

epoch 1
-------------------------------
Training on 159000 samples...
Batch size: 2867
Number of batches: 56


...

Validation Error: 
 Avg loss: 0.003281 

Learning rate (antes): 1.0000000000000028e-05
Learning rate (depois): 1.0000000000000028e-05

epoch 99
-------------------------------
Training on 159000 samples...
Batch size: 2867
Number of batches: 56


100%|██████████| 56/56 [03:58<00:00,  4.26s/it]


Training Error: 
 Avg loss: 0.003283 



100%|██████████| 19/19 [00:46<00:00,  2.43s/it]

Validation Error: 
 Avg loss: 0.003281 

Learning rate (antes): 1.0000000000000028e-05
Learning rate (depois): 1.0000000000000028e-05
CPU times: total: 7h 14min 58s
Wall time: 9h 32min 7s


In [11]:
df_f2_5 = run_srcnn.get_model_df()
df_f2_5.tail()

,epoch,time,lr,train_loss,validation_loss
95,95,339.504954,0.00001,0.003284,0.003281
96,96,343.193618,0.00001,0.003288,0.003281
97,97,350.339675,0.00001,0.003284,0.003281
98,98,341.992534,0.00001,0.003284,0.003281
99,99,347.156817,0.00001,0.003283,0.003281


In [12]:
run_srcnn.save_model_df('./results/srcnn/dataframes/model_f2_5_unfiltered.csv')
run_srcnn.save_model_weights(model_f2_1, './results/srcnn/dataframes/model_f2_5_unfiltered.pth')

In [13]:
path = f"{DATASET_ROOT}/Set14"
run_srcnn.compare_models([{"name": "model_f2_1_unfiltered", "model": model_f2_1}, {"name": "model_f2_5_unfiltered", "model": model_f2_5}], path)

0it [00:00, ?it/s]C:\Users\Pedro\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
14it [00:02,  6.77it/s]
14it [00:01,  8.60it/s]


,model_f2_1_unfiltered,model_f2_5_unfiltered
PSNR,24.604319,24.716616
Bicubic PSNR,24.496832,24.496832
SSIM,0.717399,0.725149
Bicubic SSIM,0.731670,0.731670
